# Laboratorio 1 - Regresión

Este notebook tiene los siguientes elementos: 
1. Cargue de los datos.

2. Entendimiento de los datos: Describir las características más relevantes de los datos y todo el perfilamiento de datos, incluir el análisis de calidad de datos y hacer una preselección de las variables más importantes para la etapa de modelado.

3. Preparación de datos: Solucionar los problemas de calidad de datos previamente identificados que afecten el modelo a construir. Además, debe aplicar todos los proceso de preprocesamiento de datos necesarios para la construcción del modelo de regresión.

4. Modelado: Utilizando las variables previamente seleccionadas, construir un modelo de regresión que estime la variable objetivo con el menor error posible.

5. Evaluación cuantitativa: A partir de las métricas seleccionadas para evaluar y seleccionar el mejor modelo, explicar el resultado obtenido desde el punto de vista cuantitativo. Contestar a la pregunta: ¿Su equipo recomienda utilizar en producción el modelo de regresión para estimar los tiempos? ¿Por qué? En caso de no recomendar el uso del modelo, ¿qué recomendaciones haría para continuar iterando con el objetivo de la construcción de un mejor modelo?

6. Evaluación cualitativa: Debe estar compuesta de dos partes:
- Validación de supuestos: Realizar los ajustes necesarios para que el modelo cumpla con los supuestos necesarios para la inferencia estadística con regresiones.
- Interpretación de los coeficientes: Realizar la interpretación de los coeficientes de la regresión, identificando las variables más relevantes para la estimación y cómo afectan la variable objetivo.

### Entendimiento del negocio:
El caso de estudio es de un hospital que haciendo uso de la metodología KTAS quiere solicitar un modelo que pueda pronosticar el tiempo de duración de una persona en el hospital con base en sus condiciones de llegada.
### Enfoque Analítico:
En este laboratorio vamos a hacer un modelo predictivo usando un aprendizaje supervisado y un modelo de regresión lineal para hacer uso de las condiciones de llegada de los pacientes y predecir la duración de su estancia en el hospital en minutos.

## 1. Carga de los datos

In [ ]:
import numpy as np
import pandas as pd

from joblib import dump, load

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import scipy.stats as stats
import scipy.optimize as optimize
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
import contractions
import re, string, unicodedata
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer



In [ ]:
datos = pd.read_csv("./data/Regresión_train_data.csv")

In [ ]:
datos.shape

In [ ]:
datos.dtypes

In [ ]:
datos.sample(5)

## 2. Entendimiento de los datos

Se nos proporcionaron dos CSV, uno para entrenar el modelo y otro para probarlo, al analizar los datos dados en el CSV de entrenamiento seguimos los siguientes pasos:

In [ ]:
datos.describe()

En el CSV hay datos de 1000 pacientes, con 23 características independientes.

Se puede ver que un 75% de los valores de la variable objetivo "Duracion_Estancia_Min" están sobre los 620 minutos (10 horas y 20 minutos), sin embargo, hay datos que alcanzan hasta los 709,510 minutos (1 año y 127 días).

Al ver los atributos de cada una de las filas agrupamos las características de la siguiente manera.

| Category |Fields|
|----------|------|
| Demografía |Sexo, Edad, Grupo|
| Accidente |Modo Llegada, Lesión, Queja, Principal|
| Signos Vitales |Estado Mental, SBP, DBP, HR, RR, BT, Saturación, Dolor|
| Diagnósticos |dolor NRS, KTAS enfermera, Diagnóstico En Urgencias, Disposición, KTAS experto, Duración_Estancia_Min, Duración_KTAS_Min, Error_Triaje|

Lo que nos dice es que una fila de los datos que tiene información de un paciente se compone de su demografía, accidente, los signos vitales y el diagnóstico que le dio en el ala de urgencias.
Entré las características más relevantes se encuentran:
- Los signos vitales: Estado mental, presión Arterial Sistólica, Presión Arterial Diastólica, Frecuencia cardíaca, Frecuencia Respiratoria, Temperatura Corporal y Saturación de Oxígeno.
- La edad de los pacientes.
- El Triaje realizado por la enfermera y los expertos.
- El modo en el que llevan al hospital.

Se realiza un análisis de completitud en el que podemos ver que todos los datos están completos, cuentan con 1000 registros, menos los signos vitales. El más destacado es la saturación, que un 50% de los datos no cuenta con este valor. El resto de los signos vitales no supera el 20% de faltantes. Creemos que la toma de estos signos está relacionada con la gravedad del estado del paciente.

In [ ]:
datos.isnull().sum() / datos.shape[0]

Los datos de la variable dolor_NRS están completos, no hay nulos, pero casi el 44% de los datos está marcado con #BOÞ!.

In [ ]:
datos["dolor_NRS"].value_counts()/datos.shape[0]

La Duracion_KTAS_Min se encuentra guardada como object y no como float64. Se decide realizar este cambio para poder continuar con el entendimiento de los datos.

In [ ]:
datos["Duracion_KTAS_Min"] = datos["Duracion_KTAS_Min"].str.replace(',', '.').astype('float64')

### 2.1. Búsqueda de relaciones con la variable objetivo

Si se logra visualizar o cuantificar altas correlaciones entre las variables de entrada y la variable objetivo, se podrán soportar las decisiones del experto con base en la evidencia.

Para fines prácticos se extrae en una lista todas las variables numéricas que se pueden procesar.

In [ ]:
numericas = ["Grupo", "Sexo", "Edad",  "Modo_Llegada", "Lesion", "Estado_Mental", "Dolor","KTAS_enfermera", "SBP", "DBP", "HR", "RR", "BT", "Saturacion", "Disposicion", "KTAS_experto", "Duracion_KTAS_Min", "Duracion_Estancia_Min"]

Dado que existen una gran brecha entre la duración de la estancia, que el 75% está por debajo de 700 y después de esos se dispara, tomamos la decisión de no contar con esos datos para poder seguir con el entendimiento. Como se puede ver a continuación. Los datos crecen de forma razonable hasta cierto punto.

In [ ]:
plt.plot(np.log(np.sort(datos["Duracion_Estancia_Min"]+1)))

In [ ]:
sns.boxplot(datos["Duracion_Estancia_Min"])

Para continuar con el entendimiento se decide tomar el percentil 0.8 de los datos, lo que equivale a todas las filas que el tiempo de su estancia sea menor a 858 minutos, lo que nos da unos datos menos dispersos y manejables. 

In [ ]:
val = datos["Duracion_Estancia_Min"].quantile(0.8)
datos_recorte =datos[datos["Duracion_Estancia_Min"]<=val]
sns.boxplot(datos_recorte["Duracion_Estancia_Min"])
print(datos_recorte.shape)
print(val)

Realizamos una gráfica scatter por cada variable numérica para tratar de ver tendencias de comportamiento de los datos.

In [ ]:
for variable in numericas:
    sns.pairplot(datos_recorte.sample(frac=0.2), height=3, y_vars="Duracion_Estancia_Min", x_vars=variable, kind="scatter")

Lo más notable de estos gráficos y que usaremos luego, es una posible distribución logarítmica normal de la variable de Duracion_KTAS_Min contra Duracion_Estancia_Min.

Probamos con todos los datos y su relación con la variable objetivo, se puede apreciar que ninguna de las variables, obviando "Grupo", superan el 5% de coeficiente de correlación.

In [ ]:
plt.figure(figsize=(12, 10))
cmap = sns.diverging_palette(220, 20, as_cmap=True)

sns.heatmap(
    datos[numericas].corr(),
    cmap=cmap,
    vmin=-1, vmax=1,
    annot=True
)

plt.show()

Ahora, haciendo uso del recorte, las correlaciones aumentan significativamente, sin embargo, siguen habiendo correlaciones bastante poco significativas (no superiores al 25%).

Las variables que más relaciones tienen son Grupo: 40%, Lesion: 16%, KTAS_experto: 24%, edad: 21%, la disposicion: 18% y la duración del KTAS: 20%. Los signos vitales son muy dispersos y podemos optar por unirlos.

In [ ]:
plt.figure(figsize=(12, 10))
cmap = sns.diverging_palette(220, 20, as_cmap=True)

sns.heatmap(
    datos_recorte[numericas].corr(),
    cmap=cmap,
    vmin=-1, vmax=1,
    annot=True
)

plt.show()

En la preselección de los datos elegimos las variables más relevantes con relación a la objetivo.

In [ ]:
candidatas = ["Grupo", "Lesion","KTAS_experto","Disposicion",  "Edad", "Duracion_KTAS_Min"]


## 3. Preparación de datos

Problemas de calidad:

Primero el dolor reportado por la enfermera, el cual tiene un 44% de sus entradas en null.
Nos dimos cuenta de que estas se correspondían cuando el paciente no tenía dolor, por lo que las asignamos a 0. Esto tenía una excepción en 2 entradas, las cuales registraban que el paciente tenía dolor, pero no especificaba cuál, por lo que le asignamos la media.

In [ ]:
def calida_dolor_NRS(registro):
    if registro["dolor_NRS"] == "#BOÞ!" and registro["Dolor"] == 0:
        return 0
    elif registro["dolor_NRS"] == "#BOÞ!" and registro["Dolor"] != 0:
        return registro["dolor_NRS"]
    return int(registro["dolor_NRS"])
datos['dolor_NRS'] = datos.apply(calida_dolor_NRS, axis=1)

mean = datos[datos["dolor_NRS"] != "#BOÞ!" ]['dolor_NRS'].mean()
datos["dolor_NRS"] = datos["dolor_NRS"].apply(lambda x: round(mean) if x == "#BOÞ!" else x)
datos["dolor_NRS"].value_counts()

Después usamos la técnica del One-Hot, para estandarizar el Sexo y la Lesión.

In [ ]:
datos["Sexo_stan"] = datos["Sexo"].apply(lambda x: 0 if x == 2 else x)
datos["Sexo_stan"].value_counts()

In [ ]:
datos["Lesion_stan"]=datos["Lesion"].apply(lambda x:0 if x == 2 else x)
datos["Lesion_stan"].value_counts()

Hicimos una investigación para agrupar los signos vitales en esta y llegamos al Early Warning Score (EWS) es una herramienta clínica utilizada para identificar a los pacientes que están en riesgo de deterioro. El cálculo del EWS generalmente se basa en una serie de parámetros fisiológicos como la frecuencia cardíaca, la presión arterial, la temperatura, la frecuencia respiratoria, y el nivel de conciencia, la calculamos y la agregamos a los datos.

In [ ]:
def calcularEWS(registro):
    total = 0
    
    # Frecuencia respiratoria (RR)
    if registro.get('RR', np.nan) <= 8:
        total += 2
    elif 9 <= registro.get('RR', np.nan) <= 14:
        total += 0
    elif 15 <= registro.get('RR', np.nan) <= 20:
        total += 1
    elif 21 <= registro.get('RR', np.nan) <= 29:
        total += 2
    elif registro.get('RR', np.nan) >= 30:
        total += 3
    
    # Presión arterial sistólica (SBP)
    if registro.get('SBP', np.nan) <= 70:
        total += 3
    elif 71 <= registro.get('SBP', np.nan) <= 80:
        total += 2
    elif 81 <= registro.get('SBP', np.nan) <= 100:
        total += 1
    elif 101 <= registro.get('SBP', np.nan) <= 199:
        total += 0
    elif registro.get('SBP', np.nan) >= 200:
        total += 2

    # Frecuencia cardíaca (HR)
    if registro.get('HR', np.nan) <= 40:
        total += 2
    elif 41 <= registro.get('HR', np.nan) <= 50:
        total += 1
    elif 51 <= registro.get('HR', np.nan) <= 100:
        total += 0
    elif 101 <= registro.get('HR', np.nan) <= 110:
        total += 1
    elif 111 <= registro.get('HR', np.nan) <= 129:
        total += 2
    elif registro.get('HR', np.nan) >= 130:
        total += 3

    # Temperatura corporal (BT)
    if registro.get('BT', np.nan) < 35.0:
        total += 2
    elif 35.0 <= registro.get('BT', np.nan) <= 38.4:
        total += 0
    elif registro.get('BT', np.nan) >= 38.5:
        total += 2

    # Saturación de oxígeno (Saturacion)
    if registro.get('Saturacion', np.nan) <= 91:
        total += 3
    elif 92 <= registro.get('Saturacion', np.nan) <= 93:
        total += 2
    elif 94 <= registro.get('Saturacion', np.nan) <= 95:
        total += 1

    # Nivel de conciencia
    if registro.get('Estado_Mental', np.nan) == 1:
        total += 0
    elif registro.get('Estado_Mental', np.nan) == 2:
        total += 1
    elif registro.get('Estado_Mental', np.nan) == 3:
        total += 2
    elif registro.get('Estado_Mental', np.nan) == 4:
        total += 3

    return total

datos['EWS'] = datos.apply(calcularEWS, axis=1)
datos.head()

Por último, intentamos linealizar la "Duracion_KTAS_Min".

In [ ]:
datos["Dur_KTAS_lin"] = np.log(datos["Duracion_KTAS_Min"])

In [ ]:
numericas = ["Grupo", "Sexo", "Edad",  "Modo_Llegada", "Lesion","Lesion_stan", "Estado_Mental", "Dolor","KTAS_enfermera", "EWS", "Disposicion", "KTAS_experto", "Duracion_KTAS_Min","Dur_KTAS_lin", "Duracion_Estancia_Min"]

In [ ]:
val = datos["Duracion_Estancia_Min"].quantile(0.8)
datos_recorte =datos[datos["Duracion_Estancia_Min"]<=val]
plt.figure(figsize=(12, 10))
cmap = sns.diverging_palette(220, 20, as_cmap=True)

sns.heatmap(
    datos_recorte[numericas].corr(),
    cmap=cmap,
    vmin=-1, vmax=1,
    annot=True
)

plt.show()

Usando este nuevo heatmap tenemos las siguientes variables candidatas.

In [ ]:
candidatas = ["Grupo", 
              "Edad", 
              "Disposicion",
              "Lesion_stan",
              "KTAS_experto",
              "EWS",
              "Duracion_KTAS_Min"]

In [ ]:
plt.figure(figsize=(12, 10))
cmap = sns.diverging_palette(220, 20, as_cmap=True)

sns.heatmap(
    datos_recorte[candidatas+["Duracion_Estancia_Min"]].corr(),
    cmap=cmap,
    vmin=-1, vmax=1,
    annot=True
)
plt.show()

Hacemos una limpieza de duplicados:

In [ ]:
# Se eliminan los registros totalmente duplicados
datos = datos.dropna(subset=["Duracion_Estancia_Min"]+candidatas)

In [ ]:
datos.shape

In [ ]:
total_rows = datos.shape[0]

No se hay duplicados totales.

In [ ]:
datos[["Duracion_Estancia_Min"]+candidatas].isnull().sum() / datos.shape[0]

Duplicados parciales:

In [ ]:
datos.loc[datos.duplicated(subset=candidatas, keep=False)][["Duracion_Estancia_Min"]+candidatas].head(10)

In [ ]:
duplicated_rows = datos.loc[datos.duplicated(subset=candidatas, keep=False)].shape[0]
duplicated_rows

In [ ]:
print(f"Duplicates: {(duplicated_rows/total_rows)*100:.4f}%")

In [ ]:
datos.loc[datos.duplicated(subset=candidatas+["Duracion_Estancia_Min"], keep=False)].tail(4)

In [ ]:
duplicated_rows = datos.loc[datos.duplicated(subset=candidatas+["Duracion_Estancia_Min"], keep=False)].shape[0]
duplicated_rows

In [ ]:
print(f"Duplicates: {(duplicated_rows/total_rows)*100:.4f}%")

In [ ]:
datos.drop_duplicates(subset=candidatas, inplace=True)
datos.drop_duplicates(subset=candidatas+["Duracion_Estancia_Min"], inplace=True)

In [ ]:
datos.shape

Aquí hay dos escenarios a analizar:
1. Existe 94 registros o un ~9.4% de registros con variables de entrada duplicadas, con variable objetivo diferente. Una cantidad es un poco preocupante que requieren ser limpiados para no confundir al modelo.

2. Al incluir la variable objetivo dentro del análisis de duplicados, se obtiene el ~6.7% registros duplicados adicionales. Esto es un problema potencial que obligaría al algoritmo de optimización a enfocarse más en aquellos registros duplicados.

In [ ]:
val = datos["Duracion_Estancia_Min"].quantile(0.75)
datos_recorte =datos[datos["Duracion_Estancia_Min"]<=val]

### 3.2. Particionamiento del conjunto de datos en entrenamiento y prueba

Se desea construir un modelo que se ajuste bien a los datos de entrenamiento, pero que además se comporte de forma similar con datos previamente desconocidos.

En esta parte se dividen los datos en dos conjuntos, prueba y entrenamiento. El conjunto de prueba corresponderá al 30% de los datos limpiados previamente.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(datos_recorte[candidatas], datos_recorte["Duracion_Estancia_Min"], test_size=0.3, random_state=1)

In [ ]:
X_train.shape, y_train.shape

In [ ]:
X_test.shape, y_test.shape


## 4. Modelado

Utilizando las variables previamente seleccionadas, construir un modelo de regresión que estime la variable objetivo con el menor error posible.

In [ ]:
regression = LinearRegression()
regression.fit(X_train, y_train)


## 5. Evaluación cualitativa

In [ ]:
pd.DataFrame({"columns": candidatas, "coef": regression.coef_})

In [ ]:
f, axs = plt.subplots(1, len(candidatas), sharey=True, figsize=(20, 5), layout="constrained")

for i in range(len(candidatas)):
    col = candidatas[i]
    x = X_train[col]
    m = regression.coef_[i]
    b = regression.intercept_

    axs[i].plot(x, y_train, "o", alpha=0.1)
    axs[i].plot(x, x * m + b)
    axs[i].set_title(col)
print("Train:", np.sqrt(mean_squared_error(y_train, regression.predict(X_train))))
print("Test:", np.sqrt(mean_squared_error(y_test, regression.predict(X_test))))


In [ ]:
plt.figure(figsize=(20, 3))
sns.boxplot(x=y_test, showmeans=True, orient="h")
plt.title("Valor real de $\t{Duracion Estancia Min}$ en el conjunto de prueba")
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(20, 3))
sns.boxplot(x=abs(y_test - regression.predict(X_test)), showmeans=True, orient="h")
plt.title("|Valor real - Valor estimado|/Valor estimado de $\t{Tiempo}$")
plt.xlabel("Error")
plt.grid()
plt.show()

In [ ]:
pd.DataFrame({"columns": candidatas, "coef": regression.coef_})

## 5.1 Validación de supuestos

Entre los supuestos del análisis era que:
- Debido a la convención del KTAS (entre más leve la emergencia mayor el KTAS), este tendría un coeficiente negativo con respecto a la duración del paciente en el hospital. Además de esto, podemos ver que el KTAS dado por el experto es un poco más significativo y útil que el dado por la enfermera.

- Entre más edad tiene el paciente mayor va a ser el tiempo que dura en el hospital.

- Entre peor sea la disposición de llegada del paciente, también aumenta su duración.

- Entre peor sean sus signos vitales (EWS) más tiempo tomaría su visita al hospital

## 5.2 Interpretación de los coeficientes

El modelo al que llegamos tiene un error cuadrático medio de alrededor de 130 minutos, es decir que hay un margen de error de 130 minutos entre lo que predice el modelo y el valor real que va a demorarse el paciente. Por esta razón, consideramos que el modelo no está listo para ser usado y no es aún una herramienta fiable para calcular la duración de la estancia de los pacientes.

En cuanto a los coeficientes podemos ver el peso que tienen cada una de estas variables en el tiempo de estadía del paciente:

- Grupo, en la cual un "aumento" de grupo implica un aumento de 118 minutos.

- Edad, cada año adicional del paciente implica 1 minuto adicional de espera, lo cual puede no verse como mucho, pero teniendo en cuenta que el 75% de los pacientes están sobre los 70 años, esto vuelve a la edad muy significativa.

- Disposición, el resultado de su proceso de urgencias significa 6 minutos adicionales, entre peor se considere (6 minutos para un alta a domicilio, mientras que la cirugía implica 42 minutos adicionales).

- Lesión, que el paciente tenga o no una lesión, implica que alrededor de 54 minutos de espera adicional.

- KTAS experto, cada aumento en la escala de KTAS implica menos severidad en la emergencia, lo que se traduce en 34 minutos menos de duracion en el hospital.

- EWS, el EWS es un compilado de los signos vitales y revisa si están en orden o son una, existe una amenaza para la salud del paciente, cada amenaza puede subir entre 1 y 3 puntos el EWS y cada uno de esos puntos implica 3 minutos más en la duracion.

- Duración KTAS en minutos, cada minuto de evaluación KTAS implica 40 segundos de duración adicional en la consulta. (va de 0 a 20, entonces a lo sumo implica 13.6 minutos adicionales).

## 7. Pipeline

Para la exportación del modelo definimos funciones de trasnformación que son los pasos que deben seguir los datos de entreada para que el modelo prueda procesarlo.

En la funcion de transformación objetivos de elijen las variables que se aplicarán en el modelo de regresión.

In [ ]:
         
def objetivos(X):
    X = X.copy()
    candidatas = ["grupo", "edad", "disposicion", "lesion_stan","ktas_experto","ews", "duracion_ktas_min"]
    return X[candidatas]


En la funcion de transformación limpieza se comprueba el estado de las variables de interes:
- Valores nulos: De serlo se rellena con el valor mas frecuente segun el entrenamiento
- Tipos de datos: Debes ser de tipo numerico int o float

Posterior se aplica la estarización mencioanda en la preparación de los datos.

Por ultimo se ponen todas las llaves un minuscula para evitar errores.

In [ ]:

def limpieza(X):
    X.copy()
    
    X.columns = [col.lower() for col in X.columns]
    
    X['grupo'] = X['grupo'].fillna(0).astype(int)
    
    X['ktas_experto'] = X['ktas_experto'].fillna(0).astype(int)
    
    X['edad'] = X['edad'].fillna(0).astype(int)
    
    X['disposicion'] = X['disposicion'].fillna(0).astype(int)
    
    X['lesion'] = X['lesion'].fillna(0).astype(int)
    X["lesion_stan"] = X["lesion"].apply(lambda x:0 if x == 2 else x)
        
    if X["duracion_ktas_min"].dtype == 'object': 
        X["duracion_ktas_min"] = X["duracion_ktas_min"].str.replace(',', '.').astype('float64')
    elif X["duracion_ktas_min"].dtype not in ['float64', 'int64']:  
        X["duracion_ktas_min"] = pd.to_numeric(X["duracion_ktas_min"], errors='coerce').fillna(0)
    
    
    return X


Se agrupan las variables como signos vitales en un paso adicional.

In [ ]:
def calcular_EWS_pipe(registro):
    total = 0
    
    # Frecuencia respiratoria (rr)
    if registro.get('rr', np.nan) <= 8:
        total += 2
    elif 9 <= registro.get('rr', np.nan) <= 14:
        total += 0
    elif 15 <= registro.get('rr', np.nan) <= 20:
        total += 1
    elif 21 <= registro.get('rr', np.nan) <= 29:
        total += 2
    elif registro.get('rr', np.nan) >= 30:
        total += 3
    
    # Presión arterial sistólica (SBP)
    if registro.get('sbp', np.nan) <= 70:
        total += 3
    elif 71 <= registro.get('sbp', np.nan) <= 80:
        total += 2
    elif 81 <= registro.get('sbp', np.nan) <= 100:
        total += 1
    elif 101 <= registro.get('sbp', np.nan) <= 199:
        total += 0
    elif registro.get('sbp', np.nan) >= 200:
        total += 2

    # Frecuencia cardíaca (HR)
    if registro.get('hr', np.nan) <= 40:
        total += 2
    elif 41 <= registro.get('hr', np.nan) <= 50:
        total += 1
    elif 51 <= registro.get('hr', np.nan) <= 100:
        total += 0
    elif 101 <= registro.get('hr', np.nan) <= 110:
        total += 1
    elif 111 <= registro.get('hr', np.nan) <= 129:
        total += 2
    elif registro.get('hr', np.nan) >= 130:
        total += 3

    # Temperatura corporal (BT)
    if registro.get('bt', np.nan) < 35.0:
        total += 2
    elif 35.0 <= registro.get('bt', np.nan) <= 38.4:
        total += 0
    elif registro.get('bt', np.nan) >= 38.5:
        total += 2

    # Saturación de oxígeno (Saturacion)
    if registro.get('saturacion', np.nan) <= 91:
        total += 3
    elif 92 <= registro.get('saturacion', np.nan) <= 93:
        total += 2
    elif 94 <= registro.get('saturacion', np.nan) <= 95:
        total += 1

    # Nivel de conciencia
    if registro.get('estado_mental', np.nan) == 1:
        total += 0
    elif registro.get('estado_mental', np.nan) == 2:
        total += 1
    elif registro.get('estado_mental', np.nan) == 3:
        total += 2
    elif registro.get('estado_mental', np.nan) == 4:
        total += 3

    return total

def agrupar_signos_vitales(X):
        X.copy()
        X['ews'] = X.apply(calcular_EWS_pipe, axis=1)
        return X

Se ordenan las intrucciones en el pipe line. La ultima instrucción "Regresión" corresponde a la linealización de los datos.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(datos_recorte, datos_recorte["Duracion_Estancia_Min"], test_size=0.3, random_state=1)

# Crear el pipeline
pipe = Pipeline([
    ('limpieza', FunctionTransformer(limpieza, validate=False)),
    ('Agrupacion', FunctionTransformer(agrupar_signos_vitales, validate=False)),
    ('objetivo', FunctionTransformer(objetivos, validate=False)),
    ('Regresion', LinearRegression())
])

# Entrenar el pipeline
pipe.fit(X_train, y_train).score(X_test, y_test)


Una vez armado y entrenado el modelo, se guarda en el archivo "modelo_regresion_MediAlpes" para su posterior utilización.

In [ ]:
dump(pipe, 'modelo_regresion_MediAlpes.pkl', compress=3)
print("Modelo guardado exitosamente.")

A continuacion se muestra un ejemplo de carga y ejecución del modelo

In [ ]:
candidatas = ["grupo", "lesion_stan","ktas_experto","ews", "duracion_ktas_min"]
registro_prueba = datos_recorte.sample(1)
modelo_cargado = load('modelo_regresion_MediAlpes.pkl')
y_pred = modelo_cargado.predict(registro_prueba)
print(f"Para el registro\n{registro_prueba[candidatas]}\n. Se estima una duración de la estancia de {y_pred} minutos")

## 8. Generar predicciones

Para realizar las prediciones con el modelo generado se cargan los datos y el modelo.

In [ ]:
datos_sin_etiquetas = pd.read_csv("./data/Regresión_validation_data.csv")
modelo = load('modelo_regresion_MediAlpes.pkl')

Se aplica el modelo a los datos sin etiqueta.

In [ ]:
Duracion_Est_min_prediccion = modelo.predict(datos_sin_etiquetas)
Duracion_Est_min_prediccion

Se unen las predicciones con los datos sin entiqueta.

In [ ]:
datos_predicion = pd.concat([datos_sin_etiquetas, pd.Series(Duracion_Est_min_prediccion, name = "Duracion_Estancia_Min")], axis = 1)
datos_predicion

Se guarda en un archivo los resultados de las predicciones con la llave "Duracion_Estancia_Min"

In [ ]:
datos_predicion.to_csv("./data/Regresión_predict_data.csv", sep=',', index=False, encoding='utf-8')

## Procesamiento de texto

In [ ]:
datos['Queja_Principal'].info()

In [ ]:
textos = datos.copy()
textos['Conteo'] = [len(x) for x in textos['Queja_Principal']]
textos['Max'] = [[max([len(x) for x in i.split(' ')])][0] for i in textos['Queja_Principal']]
textos['Min'] = [[min([len(x) for x in i.split(' ')])][0] for i in textos['Queja_Principal']]
textos[["Queja_Principal", "Conteo", "Max",	"Min"]]

In [ ]:

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        if word:  # Verifica que la palabra no sea None o una cadena vacía
            new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
            new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = [word.lower() for word in words if word]  # Usa comprensión de listas
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = [re.sub(r'[^\w\s]', '', word) for word in words if word]  # Usa comprensión de listas
    return [word for word in new_words if word]

#def replace_numbers(words):
#    """Replace all interger occurrences in list of tokenized words with textual representation"""
#    p = inflect.engine()
#    print(words)
#    new_words = []
#    for word in words:
#        if word.isdigit():
#            new_word = p.number_to_words(word)
#            new_words.append(new_word)
#            print("if " + new_word)
#        else:
#            new_words.append(word)
#    return new_words

def lemmatize_words(words):
    """Lemmatize list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return lemmatized_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    stop_words = set(stopwords.words('english'))  # Usa un set para búsquedas más rápidas
    filtered_words = [word for word in words if word not in stop_words]
    return filtered_words

def preprocessing(words):
    words = to_lowercase(words)
#   words = replace_numbers(words)
    words = remove_punctuation(words)
    words = remove_non_ascii(words)
    words = remove_stopwords(words)
    words = lemmatize_words(words)
    return words

In [ ]:
datos['Queja_Principal'] = datos['Queja_Principal'].apply(contractions.fix)

In [ ]:
datos['words'] = datos['Queja_Principal'].apply(lambda x: preprocessing(word_tokenize(x)))
datos['words1']=datos['words'].apply(preprocessing)
datos['words'] = datos['words1'].apply(lambda x: ' '.join(map(str, x)))
datos['words'].dropna()

datos.head()


In [ ]:
X_data, y_data = datos['words'],datos['Duracion_Estancia_Min']


In [ ]:
dummy = CountVectorizer(binary=True)
X_dummy = dummy.fit_transform(X_data)  # X_data es tu lista de textos
print(X_dummy.shape)

# Paso 2: Convertir a DataFrame (esto es opcional si solo quieres ver los datos en formato de tabla)
X_dense = X_dummy.toarray()
X_df = pd.DataFrame(X_dense, columns=dummy.get_feature_names_out())

# Paso 3: Aplicar TF-IDF usando TfidfTransformer
tt = TfidfTransformer(norm='l2', use_idf=True)
tt_matrix = tt.fit_transform(X_dummy)  # Usa X_dummy directamente

# Paso 4: Convertir a una matriz densa si es necesario para inspección o para usarla en otro proceso
tt_matrix_dense = tt_matrix.toarray()

# Paso 5: Crear DataFrame con la matriz TF-IDF
vocab = dummy.get_feature_names_out()
tt_df = pd.DataFrame(np.round(tt_matrix_dense, 2), columns=vocab)
tt_df["abdomen"].value_counts()

pd.DataFrame([df], columns=feature_names)


In [ ]:
# Convertir cada fila de la matriz en una lista
rows_as_lists = np.round(tt_matrix_dense, 2).tolist()

# Crear un DataFrame donde cada fila es una lista
tt_df = pd.DataFrame(rows_as_lists)

# Convertir las filas del DataFrame en una sola columna
tt_df = pd.DataFrame({'vector_palabras': tt_df.values.tolist()})

tt_df

In [ ]:
datos_vector = datos.reset_index()
datos_vector = pd.concat([datos_vector, tt_df], axis=1)
datos_vector

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(datos_vector, y_data, test_size=0.3, random_state=42)

In [ ]:
regressor = LinearRegression()

# Entrenar el modelo
regressor.fit(X_train, y_train)

In [ ]:
print("Train:", np.sqrt(mean_squared_error(y_train, regressor.predict(X_train))))
print("Test:", np.sqrt(mean_squared_error(y_test, regressor.predict(X_test))))

In [ ]:
regressor = LinearRegression()
# Entrenar el modelo
regressor.fit([np.array([0,1]), np.array([0,2])], [np.array([1,0]), np.array([2,0])])